# Mask-Filling task

## Import libraries

In [261]:
import pandas as pd
import numpy as np

In [262]:
import random

In [263]:
import tqdm

In [264]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [265]:
import string

In [266]:
import os

In [267]:
os.getcwd()

'c:\\Users\\Giulia\\Documents\\MyGithub\\Français_Exercises\\french-exercises\\notebooks'

## Dataset

In [268]:
df = pd.DataFrame(np.zeros((1,1)))
with open('../data/in/lepetitprinceexupery.txt', 'r') as f:
     lines = f.readlines()
     for line in lines:
            data = line.strip().split("\t")
            for el in data:
                phrases = el.strip().split(".")
                for phrase in phrases:
                    df = pd.DataFrame(np.append(df.values, [[phrase]], axis=0))

In [269]:
list_punctuation_to_remove = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~)Ã©ª'

In [238]:
def check_row_is_to_remove(row):
    # convert and remove punctuation
    new_row = row.translate(str.maketrans('', '', list_punctuation_to_remove))
    # we want it has at least 5 words.
    len_row = len(new_row.split())
    if len_row < 5:
        to_clean = True
    else:
        to_clean = False
    return [to_clean, new_row]

In [239]:
df.rename(columns = {0:'original'}, inplace = True)

In [240]:
df[['check','new']] = [[check_row_is_to_remove(el)[0],check_row_is_to_remove(el)[1]] for el in df['original']]

In [241]:
df = df.loc[df['check']== False]

In [242]:
df.head(5)

,original,check,new
9,"Lorsque j'avais six ans j'ai vu, une fois, une...",False,Lorsque j'avais six ans j'ai vu une fois une m...
10,Ã‡a reprÃ©sentait un serpent boa qui avalait ...,False,‡a reprsentait un serpent boa qui avalait un ...
11,VoilÃ la copie du dessin,False,Voil la copie du dessin
14,"On disait dans le livre :""Les serpents boas av...",False,On disait dans le livre Les serpents boas aval...
15,Ensuite ils ne peuvent plus bouger et ils dor...,False,Ensuite ils ne peuvent plus bouger et ils dor...


In [243]:
import pprint

In [244]:
df['original'].iloc[0]

'Lorsque j\'avais six ans j\'ai vu, une fois, une magnifique image, dans un livre sur la ForÃªt Vierge qui s\'appelait"Histoires VÃ©cues"'

In [245]:
df['new'].iloc[0]

"Lorsque j'avais six ans j'ai vu une fois une magnifique image dans un livre sur la Fort Vierge qui s'appelaitHistoires Vcues"

## Model

In [246]:
tokenizer = AutoTokenizer.from_pretrained("moussaKam/mbarthez")

model = AutoModelForSeq2SeqLM.from_pretrained("moussaKam/mbarthez")

Example of BART model for mask-filling task in NLP.

A french sequence to sequence pretrained model based on BART.
BARThez is pretrained by learning to reconstruct a corrupted input sentence. A corpus of 66GB of french raw text is used to carry out the pretraining.
Unlike already existing BERT-based French language models such as CamemBERT and FlauBERT, BARThez is particularly well-suited for generative tasks (such as abstractive summarization), since not only its encoder but also its decoder is pretrained.

## Decode the input

In [247]:
def f_decode_input(txt):
    input_ids = tokenizer([txt], return_tensors="pt")["input_ids"]
    logits = model(input_ids).logits
    masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
    probs = logits[0, masked_index].softmax(dim=0)
    values, predictions = probs.topk(5)
    return tokenizer.decode(predictions).split()

## Example

In [248]:
TXT = "Lorsque j'avais six ans j'ai vu, une fois, une <mask> image."

In [249]:
TXT2 = "J'ai montré mon chef-d'oeuvre aux grandes <mask> et je leur ai demandé si mon dessin leur faisait peur."

In [250]:
TXT3 = "je faisais l'expérience sur <mask> de mon dessin numéro Un"

In [251]:
for txt in [TXT, TXT2, TXT3]:
    print(f_decode_input(txt))

['image', 'fois', 'seule', 'autre', 'photo']
['personnes', 'écoles', 'et', 'heures']
['la', 'le', 'l', 'de', 'les']


## Training

In [252]:
def check_prediction(to_mask, predictions):
    list_ok = []
    list_ko = []
    predictions2 = None
    for prediction in predictions:
        prediction.translate(str.maketrans('', '', list_punctuation_to_remove))
        if len(prediction)<2:
            list_ko.append(prediction)
        elif len(prediction)< 7 and len(prediction)<(len(to_mask)-2):
            list_ko.append(prediction)
        else:
            list_ok.append(prediction)
    if list_ko:
        txt2 = "Un possible synonyme de "+to_mask+" est <mask>."
        predictions2 = f_decode_input(txt2)
        predictions2 = [el.translate(str.maketrans('', '', list_punctuation_to_remove)) for el in predictions2]
        predictions2 = [el for el in predictions2 if len(el)>1]
        list_ok.extend(predictions2)
    return list_ok

In [254]:
dict_QA = {'question' : [], 'answer' : [], 'answers' : []}

for row in tqdm.tqdm(df.iterrows(), total =df.shape[0]):
    words = row[1]['new'].split()
    rnd = random.randint(0,len(words)-1)
    to_mask = words[rnd]
    while len(to_mask)<2:
        rnd = random.randint(0,len(words)-1)
        to_mask = words[rnd]    
    words[rnd] = '<mask>'
    to_txt = ''
    for el in words:
        to_txt += el + ' '
    txt = to_txt[:-1]
    predictions = f_decode_input(txt)
    predictions = check_prediction(to_mask, predictions)
    if to_mask not in predictions:
        predictions = predictions[:-1]
        predictions.append(to_mask)
    dict_QA['question'].append(txt)
    dict_QA['answer'].append(to_mask)
    dict_QA['answers'].append(predictions)

100%|██████████| 1157/1157 [11:42<00:00,  1.65it/s]


## End

In [256]:
df['question'] = dict_QA['question']
df['answer'] = dict_QA['answer']
df['answers'] = dict_QA['answers']

In [257]:
df

,original,check,new,question,answer,answers
9,"Lorsque j'avais six ans j'ai vu, une fois, une...",False,Lorsque j'avais six ans j'ai vu une fois une m...,Lorsque j'avais six ans j'ai vu une fois une m...,la,"[le, Fort, la, un, un, appelé, le]"
10,Ã‡a reprÃ©sentait un serpent boa qui avalait ...,False,‡a reprsentait un serpent boa qui avalait un ...,‡a reprsentait un serpent boa qui avalait <mas...,un,"[un, faun, uneait]"
11,VoilÃ la copie du dessin,False,Voil la copie du dessin,Voil la copie du <mask>,dessin,"[texte</s>, unh, dessin]"
14,"On disait dans le livre :""Les serpents boas av...",False,On disait dans le livre Les serpents boas aval...,<mask> disait dans le livre Les serpents boas ...,On,"[Comme, Il, Ce, est, On]"
15,Ensuite ils ne peuvent plus bouger et ils dor...,False,Ensuite ils ne peuvent plus bouger et ils dor...,Ensuite ils ne peuvent plus bouger et <mask> d...,ils,"[dor, ils, il, dorm, donc]"
...,...,...,...,...,...,...
3217,"""Alors les grelots se changent tous en larmes !",False,Alors les grelots se changent tous en larmes,Alors les grelots se changent tous en <mask>,larmes,[larmes]
3222,C'est lÃ un bien grand mystÃ¨re,False,C'est l un bien grand myst¨re,C'est l <mask> bien grand myst¨re,un,"['un, un]"
3223,"Pour vous qui aimez aussi le petit prince, co...",False,Pour vous qui aimez aussi le petit prince com...,Pour vous qui aimez aussi le petit prince comm...,oui,"[ou, mang, oui, un]"
3229,Demandez-vous : Le mouton oui ou non a-t-il m...,False,Demandezvous Le mouton oui ou non atil mang ...,Demandezvous Le mouton oui ou non atil mang la...,vous,"[vous, Vous, quand, puis, le]"


## Save the output

In [258]:
df.drop(labels = ['original','check'], axis = 1 , inplace = True)

In [259]:
df.to_csv("../data/out/df_QA.csv", index = False)

In [272]:
os.getcwd()

'c:\\Users\\Giulia\\Documents\\MyGithub\\Français_Exercises\\french-exercises\\notebooks'

In [273]:
df_QA = pd.read_csv("../data/out/df_QA.csv", header=0)

In [277]:
answer_txt = df_QA['answers'].iloc[0][1:-1]
answer_list = answer_txt.split("'")
answer_set = set(answer_list)
answer_set.remove(", ")
answer_set.remove("")
answer_list = list(answer_set)
if len(answer_list) < 2:
    pass
# answer_set.shuffle()

In [278]:

answer_txt = df_QA['answers'].iloc[rnd][1:-1]
answer_list = answer_txt.split("'")
answer = df_QA['answer'].iloc[rnd]
right = False


In [279]:
answer

0                 la
1                 un
2             dessin
3                 On
4                ils
            ...     
1152          larmes
1153              un
1154             oui
1155            vous
1156    d'importance
Name: answer, Length: 1157, dtype: object